---
## Table of Contents<a id="toc"></a>
* [Introduction](#intro)
    * [Drying](#dry)
    * [Calcination](#calcin)
* [Problem Statment](#prob)
* [Objectives](#obj)
* [Source](#src)
* [Tools](#tools)
* [Approach and Dynamics Equations](#app)
* [Challenges & Course Corrections](#correct)
* [Initialization of Solvent Extraction Module](#IPM)
* [Startup](#su)
* [Steady-State](#ss)
* [Disturbance](#malfunction)
* [References](#refs)
---

## [Introduction](#toc)<a id="intro"></a>

The incoming ammonium diuranate (ADU) slurry produced by precipitation now has to undergo a drying and calcination process at extremely high temperatures. While the centrifuge in the precipitation process drove out most of the water, it needs to be completely evaporated to obtain the final product. The product being fed into the drying and calcination section of the milling process mainly contains ammonium diuranate. This ammonium diuranate undergoes dehydration and oxidation to produce the coveted triuranium octoxide. These processes take place in a multi hearth furnace. A diagram of one of these furnaces can be found in Figure 1.

<img src="pics/multi-hearth-furnace.jpg" alt="Figure 1: Multi Hearth Furnace" width="500">


### [Drying](#toc)<a id="dry"></a>

The drying part of this process involves the removal of moisture and volatile components from the ADU slurry. This converts the slurry into a dry powder and drives off ammonia and other impurities that result from the drying. This ensures that the intermediate product is clean and stable.


</br>
<center>H<sub>2 </sub>O (a) → H<sub>2 </sub>O (v) </center>

### [Calcination](#toc)<a id="calcin"></a>

Calcination is the oxidation of the ore concentrate. Calcination happens at extremely high temperatures so this process is able to eliminate water completely and drive off any of the remaining volitile components, but care must be taken not to sinter the product as it can significantly reduce nitric acid solubility, which is critical for some refining processes. The result is the yellowcake powder which is then packaged into drums for shipping.

The chemical reaction that occurs during calcination can be summarized as follows.

</br>
<center>9(NH<sub>4</sub>)(a)<sub>2</sub>U<sub>2</sub>0<sub>7</sub>(a) → 14NH<sub>3</sub>(a) + 15H<sub>2</sub>O(v) + 2N<sub>2</sub>(v) + 6U<sub>3</sub>O<sub>8</sub>(s)</center>

SOURCES
hearth dryers: https://patentimages.storage.googleapis.com/94/7d/7c/3c2de05ed45669/US5957064.pdf

### [Problem Statement](#toc)<a id="prob"></a>

In [4]:
from cortix import ReactionMechanism

In [5]:
rxn_mech = ReactionMechanism(file_name='triuranium-octoxide-production-mechanism.txt', order_species=True)
rxn_mech.md_print()

5 **Species:** 
 ${\mathrm{H}_{2}\mathrm{O}}_\mathrm{(v)} \quad  {\mathrm{N}_{2}}_{\mathrm{(v)}} \quad  {\mathrm{N}\mathrm{H}_{3}}_{\mathrm{(v)}} \quad  {\mathrm{U}_{3}\mathrm{O}_{8}}_{\mathrm{(s)}} \quad  {[\mathrm{N}\mathrm{H}_{4}]_{2}\mathrm{U}_{2}\mathrm{O}_{7}}_{\mathrm{(a)}}$

1 **Reactions:** 
 \begin{align*} 
9.0\,{[\mathrm{N}\mathrm{H}_{4}]_{2}\mathrm{U}_{2}\mathrm{O}_{7}}_{\mathrm{(a)}}\ &\longrightarrow \ 15.0\,{\mathrm{H}_{2}\mathrm{O}}_\mathrm{(v)}\ + \ 2.0\,{\mathrm{N}_{2}}_{\mathrm{(v)}}\ + \ 14.0\,{\mathrm{N}\mathrm{H}_{3}}_{\mathrm{(v)}}\ + \ 6.0\,{\mathrm{U}_{3}\mathrm{O}_{8}}_{\mathrm{(s)}}\\ 
\end{align*} 


In [3]:
'''Data Check'''
print('Is mass conserved?', rxn_mech.is_mass_conserved())
#rxn_mech.rank_analysis(verbose=True, tol=1e-8)
#print('S=\n', rxn_mech.stoic_mtrx)

Is mass conserved? True


## [Objectives](#toc)<a id="obj"></a>

- Python code will be used to develop a dynamic comprehensive system that accurately represents the evaporation and calcination parts of the milling process as well as their role in simulating a steady state reactor. This involves both mathematical models of the system’s material streams and visual representations of the data on a time dependent basis. 
- Modules to model evaporation and calcination steps will be developed.
- Deliverable content will be prepared using Jupyter notebooks to seamlessly integrate working code with descriptive markdown text.  
- A flowsheet based on White Mesa will be developed as the basis for our modules
- Pick up where the previous group left off.


## [Tools](#toc)<a id="tools"></a>

## [Approach and Dynamics Equations](#toc)<a id="app"></a>

-  First, a series of mass balances and component balances are made based on the desired input of 1 ton of raw material.
- These balances then take into account the chemical kinetics and related rates required in the operation of White Mesa's chemical reactions and fluid flow operations
- Once baseline models of each individual process exist, they are then programmed into 5 separate but connected modules within Cortix
- Each process will be capable of interacting with the modules connected to it on either end, allowing them to share data such as the incoming and outgoing streams from one process to the next

## [Challenges & Course Correctons](#toc)<a id="correct"></a>

Initially, this section was called Evaporation and Calcination. There were modules defined for evaporation and even a falling film evaporator was proposed fro this process. After further digging, it was determined that the more accurate representation of this step would be Drying and Calcination. Although in the drying section, evaporation of water occurs, 
This was a major change and it led to a reconstruction of not only this project report, but the source code and usecase. 

## [Initialization of Process Modules](#toc)<a id="IPM"></a>

### [Evaporation/Calcining](#toc)<a id="evap"></a>

```python
                                              |
                                              |  Ammonium Diuranate Feed
                                              v
                                       ________________       
                                      |                |
                                      |                |
                                      |                |<------ Steam Sparging
                                      |     Drying     |
                                      |                |
                   Off-Gas <----------|                |<------ Resistance Heating (Internal)
                                      |                |
                                      |----------------|
                      Off-Gas <-------|                |<------ Sweeping Gas
                                      |                |
                                      |   Calcination  |
                 Sweeping Gas <-------|                |
                   (Disposal)         |                |
                                      |________________|<------ Resistance Heating (Internal)
                                               |
                                               |
                                               |
                                               v
                                        Product (U3O8)
```

__Uranium Drying Overview__

The incoming ammonium diuranate slurry produced by precipitation now has to undergo a drying process at extremely high temperatures. While the centrifuge in the precipitation process drove out most of the water, it needs to be completely evaporated to obtain the final product. The product being fed into the drying section of the milling process mainly contains ammonium diuranate, which is dried to produce the coveted triuranium octoxide. The entire drying process can be split up into two stages; drying, which is the thermal removal of water from the ammonium diurante, and calcination, which is the chemical conversion of ammonium diuranate to U3O8. 

There are two reactions that will happen based off each of the stages. In the drying stage, heat will be applied to the ammonium diurante slurry, evaporating the water and separating the water vapor from the slurry. This equation is given as:


During the calcination stage, the slurry is hit with a much greater amount of heat, chemically changing the compostion of the ammonium diuranate to the coveted triuranium octoxide product. This reaction is given as:
9(NH4)2U2O7(l)  14NH3 + 15H2O + 2N2 + 6U3O8

As a whole, this reaction can be summarized as the following:

*Production of Triuranium Octoxide*

</br>
<center>9(NH<sub>4</sub>)<sub>2</sub>U<sub>2</sub>0<sub>7</sub> → 14NH<sub>3</sub> + 15H<sub>2</sub>O + 2N<sub>2</sub> + 6U<sub>3</sub>O<sub>8</sub></center>

__Calcination Process__

The second stage of the drying process is known as calcination. In terms of the overall process it will follow a similar basis to that of evaporation, but at a higher temperature. However, there are some distinguishable differences in calcination since the feed is now mostly a pure solid. Calcination involves the heating of solids to a high temperature for the purpose of removing volatile substances, oxidizing a portion of mass, or rendering them friable. Therefore, this stage in the drying process is sometimes considered a process of absolute purification.

This stage is not similar to the "evaporation" stage because it is a whole different piece of equipment at a much larger temperature. 

This production happens in large molar quantities, and assuming a 100% conversion rate every 100 kgs of ammonium diuranate will result in 90 kgs of triuranium octoxide. High temperature calcining is practiced producing a greater than 94% U<sub>3</sub>O<sub>8</sub> product having high density [3].

Calcination eliminates water completely, and drives off the ammonia, but care must be taken not to sinter the product.

Sintering can significantly reduce nitric acid solubility, which is critical for some refining processes. Several types of drying equipment have been used in the calcining process, typically arranged in the series of dryers that succeed the main calciner tank. These include single or multiple hearth dryers, drum dryers, belt dryers, screw dryers and radiant heat dryers. There also becomes the problem of how to dispose of the ammonia used as a "sweep-gas" to drive the heat transfer in the calciner. As seen in figure 4, the solution to the produced ammonia is to add a form of fluid bed dryer to the calciner tank to extract the ammonia vapor.

__Sizing and Cost__

The size of each of the main columns used in this process will prove to be vital to the overall drying module. For evaporation, it was determined that there would be 2 columns used in this process. Each of these columns would be sized at 200m<sup>3</sup>. For calcination, it was determined that 2 MHF units would be utilized. Each of these calciner unites would have an overall volume of 350m<sup>3</sup>.

The overall cost of this operation will be strictly based on the main unit operations involved in the process. Thus, it will mainly pertain to the evapotator and calciner columns. For the evaporator tank, it will utilize a stainless steel material to withstand the high temperatures the system is going to be exposed to. Using (x), it is assumed that the cost of each of the 200m<sup>3</sup> falling film evaporator columns using this material will be 28,700 dollars. For the calciner, it will also utilize stainless steel, but will also include an interior refactory brick layer since it will have a low thermal conductivity during the heat reaction. This in turn will not only lead to protection of the stainless steel outer layer, but will also yield greater energy efficiency for the overall heating process. Using (x), each of the MHF calciner units, with a capacity of 350m<sup>3</sup>, will cost 75,370 dollars. 

In total, the overall drying process will cost approximately 208,140 dollars to construct and design.

__Modeling the Module__

To model this process, the process needs to remain seperated by the two main stages of drying. Thus, two seperate subsections to this module will be created for both evaporation and calcination.

Starting with evaporation, this basis of the model needed some input parameters to work with. The feed mass flowrate was determined to be 5800 kg/min, along with a  set overall volume of 400m<sup>3</sup> (remember there are two tanks). The third and final parameter that will be monitored is the product mass density, which is being feed in at 

a paramter had to be selected to base the model after that would be able to monitor the amount of flow entering and exiting the tank. This was chosen to be the liquid volume of the evaporator tank, but it came with some conflicts. Being able to differentiate whether the tank was filling, evaporating, or discharging into the next stage was difficult to determine using only this parameter. 
